# Prediction of energy consumption of non-residential buildings in San Francisco

July 2019

*Project Description:*

We will use this dataset to analyse and predict the Froth floating process having the two aims:

What is the best predictor for the iron concentration of the product?
Can the data set be used to predict the impurity of the product (by silicate concentration)?

*Data Description:*

This notebook deals with the analysis of a reverse cationic flotation process of a real production environment. The data (including its documentation) is accessible through kaggle: https://www.kaggle.com/san-francisco/sf-commercial-buildings-energy-performance-report/downloads/sf-commercial-buildings-energy-performance-report.zip/104

---

## Data Analysis
We start our analysis by importing required libraries:

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from IPython.display import display
from sklearn import metrics

# include fasti.ai libraries
from fastai.tabular import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from IPython.display import display
pd.set_option('display.max_columns', None) # display all columns

# Any results you write to the current directory are saved as output.

['pred_Energy', '.DS_Store']


In [9]:
PATH = 'existing-commercial-buildings-energy-performance-ordinance-report.csv'
df = pd.read_csv(PATH, parse_dates = True)

Let's have a look at the data:

In [25]:
df.head()

,Parcel(s),Building Name,Building Address,Postal Code,Full.Address,Floor Area,Property Type,Property Type - Self Selected,PIM Link,Year Built,Energy Audit Due Date,Energy Audit Status,Benchmark 2018 Status,2018 Reason for Exemption,Benchmark 2017 Status,2017 Reason for Exemption,Benchmark 2016 Status,2016 Reason for Exemption,Benchmark 2015 Status,2015 Reason for Exemption,Benchmark 2014 Status,2014 Reason for Exemption,Benchmark 2013 Status,2013 Reason for Exemption,Benchmark 2012 Status,2012 Reason for Exemption,Benchmark 2011 Status,2011 Reason for Exemption,Benchmark 2010 Status,2010 Reason for Exemption,2018 ENERGY STAR Score,2018 Site EUI (kBtu/ft2),2018 Source EUI (kBtu/ft2),2018 Percent Better than National Median Site EUI,2018 Percentage Better than National Median Source EUI,2018 Total GHG Emissions (Metric Tons CO2e),2018 Total GHG Emissions Intensity (kgCO2e/ft2),2018 Weather Normalized Site EUI (kBtu/ft2),2018 Weather Normalized Source EUI (kBtu/ft2),2017 ENERGY STAR Score,2017 Site EUI (kBtu/ft2),2017 Source EUI (kBtu/ft2),2017 Percent Better than National Median Site EUI,2017 Percentage Better than National Median Source EUI,2017 Total GHG Emissions (Metric Tons CO2e),2017 Total GHG Emissions Intensity (kgCO2e/ft2),2017 Weather Normalized Site EUI (kBtu/ft2),2017 Weather Normalized Source EUI (kBtu/ft2),2016 ENERGY STAR Score,2016 Site EUI (kBtu/ft2),2016 Source EUI (kBtu/ft2),2016 Percent Better than National Median Site EUI,2016 Percentage Better than National Median Source EUI,2016 Total GHG Emissions (Metric Tons CO2e),2016 Total GHG Emissions Intensity (kgCO2e/ft2),2016 Weather Normalized Site EUI (kBtu/ft2),2016 Weather Normalized Source EUI (kBtu/ft2),2015 ENERGY STAR Score,2015 Site EUI (kBtu/ft2),2015 Source EUI (kBtu/ft2),2015 Percent Better than National Median Site EUI,2015 Percentage Better than National Median Source EUI,2015 Total GHG Emissions (Metric Tons CO2e),2015 Total GHG Emissions Intensity (kgCO2e/ft2),2015 Weather Normalized Site EUI (kBtu/ft2),2015 Weather Normalized Source EUI (kBtu/ft2),2014 ENERGY STAR Score,2014 Site EUI (kBtu/ft2),2014 Source EUI (kBtu/ft2),2014 Percent Better than National Median Site EUI,2014 Percent Better than National Median Source EUI,2014 Total GHG Emissions (Metric Tons CO2e),2014 Total GHG Emissions Intensity (kgCO2e/ft2),2014 Weather Normalized Site EUI (kBtu/ft2),2014 Weather Normalized Source EUI (kBtu/ft2),2013 ENERGY STAR Score,2013 Site EUI (kBtu/ft2),2013 Source EUI (kBtu/ft2),2013 Percent Better than National Median Site EUI,2013 Percent Better than National Median Source EUI,2013 Total GHG Emissions (Metric Tons CO2e),2013 Total GHG Emissions Intensity (kgCO2e/ft2),2013 Weather Normalized Site EUI (kBtu/ft2),2013 Weather Normalized Source EUI (kBtu/ft2),2012 ENERGY STAR Score,2012 Site EUI (kBtu/ft2),2012 Source EUI (kBtu/ft2),2012 Percent Better than National Median Site EUI,2012 Percent Better than National Median Source EUI,2012 Total GHG Emissions (Metric Tons CO2e),2012 Total GHG Emissions Intensity (kgCO2e/ft2),2012 Weather Normalized Site EUI (kBtu/ft2),2012 Weather Normalized Source EUI (kBtu/sq.ft),2011 ENERGY STAR Score,2011 Site EUI (kBtu/ft2),2011 Source EUI (kBtu/ft2),2011 Percent Better than National Median Site EUI,2011 Percent Better than National Median Source EUI,2011 Total GHG Emissions (Metric Tons CO2e),SF Find Neighborhoods,2011 Total GHG Emissions Intensity (kgCO2e/ft2),Current Police Districts,2011 Weather Normalized Site EUI (kBtu/ft2),Current Supervisor Districts,Analysis Neighborhoods,2011 Weather Normalized Source EUI (kBtu/ft2),:@computed_region_f58d_8dbm,:@computed_region_vtsz_7cme,:@computed_region_rxqg_mtj9,:@computed_region_jx4q_fizf
0,5370/054,225 INDUSTRIAL ST,225 INDUSTRIAL ST,94124,"{'longitude': '-122.405396', 'latitude': '37.7...",11200,Commercial,Other,{'url': 'http://propertymap.sfplanning.org/?&s...,1957.0,2014-04-01T00:00:00.000,Did Not Comply,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN,V

In [29]:
df.columns[4]

'Full.Address'

Now, we will drop some columns that dont seem to be useful for our analysis:

In [32]:
df_drop = df.drop(df[df.columns[4]],inplace = True)

KeyError: '[\'{\\\'longitude\\\': \\\'-122.405396\\\', \\\'latitude\\\': \\\'37.738345\\\', \\\'human_address\\\': \\\'{"address": "225 INDUSTRIAL ST", "city": "SAN FRANCISCO", "state": "CA", "zip": "94124"}\\\'}\'\n \'{\\\'longitude\\\': \\\'-122.403285\\\', \\\'latitude\\\': \\\'37.739578\\\', \\\'human_address\\\': \\\'{"address": "50 Dorman Ave", "city": "SAN FRANCISCO", "state": "CA", "zip": "94124"}\\\'}\'\n \'{\\\'longitude\\\': \\\'-122.39278\\\', \\\'latitude\\\': \\\'37.777164\\\', \\\'human_address\\\': \\\'{"address": "255 King St", "city": "SAN FRANCISCO", "state": "CA", "zip": "94107"}\\\'}\'\n \'{\\\'longitude\\\': \\\'-122.438211\\\', \\\'latitude\\\': \\\'37.743846\\\', \\\'human_address\\\': \\\'{"address": "5290 Diamond Heights Blvd", "city": "SAN FRANCISCO", "state": "CA", "zip": "94131"}\\\'}\'\n ...\n \'{\\\'longitude\\\': \\\'-122.388989\\\', \\\'latitude\\\': \\\'37.750152\\\', \\\'human_address\\\': \\\'{"address": "1100 CESAR CHAVEZ ST", "city": "SAN FRANCISCO